Re-run the multi-catchment processing scheme to aggregate data to catchment means.  TODO: load in new dEBM data and process calibration period and forward projections.

In [ ]:
from shapely.geometry import MultiPoint
from shapely.ops import triangulate
import shapefile
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import pyproj as pyproj
from scipy import interpolate
import datetime
import time

In [ ]:
###------------------------
### DATA READ-IN  AND PROJECTION
###------------------------

## Read in BedMachine grid to reproject SMB
gl_bed_path ='/Users/lizz/Documents/GitHub/Data_unsynced/BedMachine-Greenland/BedMachineGreenland-2017-09-20.nc'
fh = Dataset(gl_bed_path, mode='r')
xx = fh.variables['x'][:].copy() #x-coord (polar stereo (70, 45))
yy = fh.variables['y'][:].copy() #y-coord
M = fh.variables['mask'][:].copy() ## land cover type mask (2 = grounded ice)
fh.close()

## Read in Mouginot catchments from shapefile
print('Reading in Mouginot catchments')
catchment_fn = '/Users/lizz/Documents/GitHub/Data_unsynced/Greenland-catchments-Mouginot/Greenland_Basins_PS_v1.4.2.'
sf = shapefile.Reader(catchment_fn) 

## Example SMB field read in for grid
print('Reading in example SMB field')
nhm_smb_path = '/Volumes/GoogleDrive/My Drive/SMBMIP/NHM-SMAP_niwano-monthly-ERA-Interim-1980.nc'
fh2 = Dataset(nhm_smb_path, mode='r')
xlon_nhm = fh2.variables['LON'][:].copy() #x-coord (latlon)
ylat_nhm = fh2.variables['LAT'][:].copy() #y-coord (latlon)
fh2.close()

In [ ]:
###------------------------
### SET UP SMB REPROJECTION
###------------------------

## Down-sample bed topo
x_3km = xx[::20] # sample at ~3 km resolution
y_3km = yy[::20]
Xmat, Ymat = np.meshgrid(x_3km, y_3km) # Downsampled BedMachine coords

## Down-sample SMB
x_lon_h = xlon_nhm[::2, ::2] 
y_lat_h = ylat_nhm[::2, ::2] # resolution about 2 km

print('Creating reprojected grid')
wgs84 = pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by SMB data
psn_gl = pyproj.Proj("+init=epsg:3413") # Polar Stereographic North used by BedMachine and Mankoff
xs, ys = pyproj.transform(wgs84, psn_gl, x_lon_h, y_lat_h)

In [ ]:
###------------------------
### CHOOSE CATCHMENTS
###------------------------
catchments_to_pull = np.arange(210, 260)

In [ ]:
###------------------------
### CREATE FRAMEWORK
###------------------------

caps_and_disjoint = []

## Perform Delaunay triangulation over each catchment region
tri_ctmts = {i: [] for i in catchments_to_pull}
for i in catchments_to_pull:
    print('Triangulating catchment {}'.format(sf.record(i)['NAME']))
    c = MultiPoint(sf.shape(i).points)
    tris = triangulate(c)
    tri_ctmts[i] = tris
    if 'ICE_CAPS' in sf.record(i)['NAME']: ## catch the ice caps and weirdos after ID 200
        caps_and_disjoint.append(i)
    elif len(sf.shape(i).parts)>1:
        caps_and_disjoint.append(i) 

# ## Perform Delaunay triangulation over each catchment region
# tri_ctmts = {i: [] for i in catchments_to_pull}
# for i in catchments_to_pull:
#     print('Triangulating catchment {}'.format(sf.record(i)['NAME']))
#     c = MultiPoint(sf.shape(i).points)
#     tris = triangulate(c)
#     tri_ctmts[i] = tris

## Create data frames to store per-model time series
model_names = ['ANICE-ITM_Berends', 'CESM_kampenhout', 'dEBM_krebs','HIRHAM_mottram', 
                'NHM-SMAP_niwano', 'RACMO_noel', 'SNOWMODEL_liston']
# model_names = ['ANICE-ITM_Berends',]
years = range(1980,2013)
start_date = datetime.datetime(years[0],1,1)
end_date = datetime.datetime(years[-1],12,31)
dates = pd.date_range(start=start_date, end=end_date, freq='M')
df_per_ctchmnt = {i: pd.DataFrame(columns=model_names, index=dates) for i in catchments_to_pull}

In [ ]:
caps_and_disjoint

In [ ]:
###------------------------
### CATCHMENT-SUM FOR ALL MODELS
###------------------------

for m in model_names:
    t0 = time.time()
    if m=='CESM_kampenhout':
        vname = 'SMBCORR'
    else:
        vname = 'SMBcorr'
    for y in years:
        ti = time.time()
        fpath = '/Volumes/GoogleDrive/My Drive/Greenland-data/SMBMIP/{}-monthly-ERA-Interim-{}.nc'.format(m, y)
        attempts = 0 ## allow multiple attempts for connectivity issues
        while attempts<=5:
            try:
                fh = Dataset(fpath, mode='r')
                smb_m = fh.variables[vname][:].copy()
                fh.close()
                break
            except RuntimeError:
                print('RuntimeError encountered, waiting in case of connectivity issue...')
                time.sleep(5) ## wait 5 seconds and try again
                attempts+=1
        d_subset = [d for d in dates if d.year==y]
        for i in range(len(smb_m)): # for each month
            ## downsample and interpolate SMB
            smb_ds = smb_m[i][::2, ::2]
            regridded_smb = interpolate.griddata((xs.ravel(), ys.ravel()), smb_ds.ravel(), (Xmat, Ymat), method='nearest')
            ## Sample SMB at each Delaunay triangle and sum
            for j in catchments_to_pull:
                if j in caps_and_disjoint:
#                 print('Identified catchment {} in ice caps and disjoint catchments. Skipping.'.format(j))
#                 df_record.drop(columns=j, inplace=True)
                    pass
                else:
                    catchment_sum = 0
                    area_sum = 0
                    triangles = tri_ctmts[j]
                    for tri in triangles:
                        rep_x, rep_y = tri.representative_point().x, tri.representative_point().y
                        area_m2 = tri.area
                        smb_x = (np.abs(x_3km - rep_x)).argmin()
                        smb_y = (np.abs(y_3km - rep_y)).argmin()
                        nearest_smb = regridded_smb[smb_y, smb_x]
                        if nearest_smb < -6000: ## catch fill values or unrealistic extreme values
                            next_smb_x = np.argsort(np.abs(x_3km-rep_x))[1]
                            next_smb_y = np.argsort(np.abs(y_3km-rep_y))[1]
                            next_nearest_smb = regridded_smb[next_smb_y, next_smb_x]
                            ### TODO: add an option for a fill value, if next nearest is also bad?
                            local_val = next_nearest_smb*area_m2
                        elif nearest_smb > 6000: ## HIRHAM and SNOWMODEL have positive fill val
                            next_smb_x = np.argsort(np.abs(x_3km-rep_x))[1]
                            next_smb_y = np.argsort(np.abs(y_3km-rep_y))[1]
                            next_nearest_smb = regridded_smb[next_smb_y, next_smb_x]
                            ### TODO: add an option for a fill value, if next nearest is also bad?
                            local_val = next_nearest_smb*area_m2
                        else:
                            local_val = nearest_smb*area_m2
                        catchment_sum += local_val
                        area_sum += area_m2
                    df_per_ctchmnt[j][m][d_subset[i]] = catchment_sum/area_sum
        tf = time.time()
        print('Finished processing year {} in time {}s'.format(y, tf-ti))
    t1 = time.time()
    print('Finished processing model {} in time {}'.format(m, t1-t0))

## Write to CSV
for i in catchments_to_pull:  
    csv_ctchmnt = '/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP-processed/{}-catchment_{}_mean-tseries.csv'.format(datetime.date.today().strftime('%Y%m%d'), i)
    df_per_ctchmnt[i].to_csv(csv_ctchmnt)
    